## Import data


In [ ]:
import csv

# Path to your CSV file
csv_file_path ='/content/Copy of AI Project Data Training Sheet - Data Sheet.csv'

# Dictionary to hold the data
data_dict = {}

# Reading the CSV file
with open(csv_file_path, mode='r', encoding='utf-8') as file:
    csv_reader = csv.DictReader(file)
    headers = csv_reader.fieldnames
    for header in headers:
        data_dict[header] = []
    for row in csv_reader:
        for header in headers:
            data_dict[header].append(row[header])

# Now `data_dict` contains all the data from the CSV file organized by column headers
print(data_dict)


{'Date ': ['05/04/2024', '21/04/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024', '17/05/2024'], 'Reference URL': ['https://www.instagram.com/reel/C5Gj8oMrOkd/?igsh=ZjF2eXhkenUwYXB0', 'https://www.instagram.com/reel/C5_wpqASpXA/?igsh=MXZqOGN3MGNqempzZw==', 'https://www.instagram.com/reel/CxsrdLjuoO5/?igsh=NGFnaHYyeDVoNjdi', 'https://www.instagram.com/reel/C1fL7j4RSfP/?igsh=MWt1MTd0dnAxc2duag==', 'https://www.instagram.com/reel/CxaY7XQOIUq/?igsh=MWpoeTIxMWE3MzUxNg==', 'https://ww

In [ ]:
import random

def split_data(data_dict, split_ratio=0.8):
    # Calculate the size of the dataset and the split index
    total_size = len(data_dict["Hook"])
    train_size = int(total_size * split_ratio)

    # Generate shuffled indices
    indices = list(range(total_size))
    random.shuffle(indices)

    # Split indices into training and testing sets
    train_indices = indices[:train_size]
    test_indices = indices[train_size:]

    # Function to create a dataset from indices
    def create_dataset(indices):
        return {
            "Hook": [data_dict["Hook"][i] for i in indices],
            "Build Up": [data_dict["Build Up"][i] for i in indices],
            "Body": [data_dict["Body "][i] for i in indices],
            "CTA": [data_dict["CTA"][i] for i in indices]
        }

    # Create training and testing datasets
    train_dataset = create_dataset(train_indices)
    test_dataset = create_dataset(test_indices)

    return train_dataset, test_dataset


In [ ]:
# Assuming data_dict is your loaded dataset
train_dataset, test_dataset = split_data(data_dict, split_ratio=0.8)

# You can print the size of each split to verify
print(f"Training set size: {len(train_dataset['Hook'])}")
print(f"Testing set size: {len(test_dataset['Hook'])}")

Training set size: 34
Testing set size: 9


## Load the Model

In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
    "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


Unsloth: You passed in `unsloth/Phi-3-mini-4k-instruct` and `load_in_4bit = True`.
We shall load `unsloth/Phi-3-mini-4k-instruct-bnb-4bit` for 4x faster loading.


config.json:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


## Add LoRA adapters

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
alpaca_prompt = """Below is a hook and a buildup that describes the context for a social media post. Based on this context, generate an engaging and elaborate body that captures the viewer's attention, provides relevant details, and encourages interaction.

Your response should aim for a minimum of 3-4 sentences and cover the following aspects:

1. Expand on the context provided by the hook and buildup, adding more details or background information to set the scene.
2. Incorporate persuasive language, rhetorical questions, or calls to action to encourage viewer engagement.
3. Consider including personal anecdotes, relatable experiences, or emotional appeals to connect with the audience.
4. Conclude with a strong closing statement that reinforces the main point or desired action.

### Hook:
{}

### Buildup:
{}

### Body:
{}"""


EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(data_dict):
    # Retrieve lists from the dictionary
    hooks = data_dict["Hook"]
    buildups = data_dict["Build Up"]
    bodies = data_dict["Body"]

    # Initialize the list for formatted texts
    formatted_texts = []

    # Iterate over items
    for hook, buildup, body in zip(hooks, buildups, bodies):
        # Format each entry and append EOS_TOKEN
        text = alpaca_prompt.format(hook, buildup, body) + EOS_TOKEN
        formatted_texts.append(text)

    return {"text": formatted_texts}


# Suppose your dataset dictionary is named data_dict
formatted_traindataset = formatting_prompts_func(train_dataset)


In [ ]:
# Example to print the first few formatted entries
for text in formatted_traindataset["text"][:5]:
    print(text)


Below is a hook and a buildup that describes the context for a social media post. Based on this context, generate an engaging and elaborate body that captures the viewer's attention, provides relevant details, and encourages interaction.

Your response should aim for a minimum of 3-4 sentences and cover the following aspects:

1. Expand on the context provided by the hook and buildup, adding more details or background information to set the scene.
2. Incorporate persuasive language, rhetorical questions, or calls to action to encourage viewer engagement.
3. Consider including personal anecdotes, relatable experiences, or emotional appeals to connect with the audience.
4. Conclude with a strong closing statement that reinforces the main point or desired action.

### Hook:
"You shouldn't have 12 employees for a 50,000-month business. You should have one, you."

### Buildup:
Build Up:
"I'll give you examples. My agency model, absolute mess, so complicated. We had 12 employees doing 50k a 

In [ ]:
from datasets import Dataset

# Assuming formatted_traindataset is your list of dictionaries
train_dataset1 = Dataset.from_dict(formatted_traindataset)


## Train the Model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset1,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/34 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
2.283 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 34 | Num Epochs = 15
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 29,884,416


Step,Training Loss
1,2.202100
2,2.086200
3,1.971900
4,2.058700
5,1.864500
6,1.854600
7,1.723400
8,1.629800
9,1.628100
10,1.559800


## Inference

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Here's how to turn 1 shitty content idea into 18 viral videos.", # ihook
        "And if you're questioning my competence, I grew my Instagram from 0 to 200K followers in six months by using this strategy.", # buildup
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 512, no_repeat_ngram_size=5, use_cache = True, early_stopping=True)

<s> Below is a hook and a buildup that describes the context for a social media post. Based on this context, generate an engaging and elaborate body that captures the viewer's attention, provides relevant details, and encourages interaction.

Your response should aim for a minimum of 3-4 sentences and cover the following aspects:

1. Expand on the context provided by the hook and buildup, adding more details or background information to set the scene.
2. Incorporate persuasive language, rhetorical questions, or calls to action to encourage viewer engagement.
3. Consider including personal anecdotes, relatable experiences, or emotional appeals to connect with the audience.
4. Conclude with a strong closing statement that reinforces the main point or desired action.

### Hook:
Here's how to turn 1 shitty content idea into 18 viral videos.

### Buildup:
And if you're questioning my competence, I grew my Instagram from 0 to 200K followers in six months by using this strategy.

### Body:
Fi

## Evaluation on train_dataset

In [ ]:
generated_bodies = []
# Assuming test_dataset = {'hook': [...], 'buildup': [...], 'body': [...]}
for hook, buildup, actual_body in zip(train_dataset['Hook'], train_dataset['Build Up'], train_dataset['Body']):
    # Prepare the prompt
    prompt = alpaca_prompt.format(
        hook,  # Hook part
        buildup,  # Buildup part
        ""  # Leave the body part empty for generation
    )

    # Tokenize the inputs
    inputs = tokenizer(
        [prompt],
        return_tensors="pt"
    ).to("cuda")

    # Generate the output using the model
    generated_outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        no_repeat_ngram_size=5,
        use_cache=True,
        early_stopping=True
    )

    # Decode the generated text
    generated_body = tokenizer.decode(generated_outputs[0], skip_special_tokens=True)
    start = generated_body.find("### Body:") + len("### Body:")
    body_text = generated_body[start:].strip()  # Extract the body text
    generated_bodies.append(body_text)

    # Print both generated and actual body for comparison
    print("Generated Body:", generated_body)
    print("Actual Body:", actual_body)
    print("\n")  # Adds a newline for better separation of entries


Generated Body: Below is a hook and a buildup that describes the context for a social media post. Based on this context, generate an engaging and elaborate body that captures the viewer's attention, provides relevant details, and encourages interaction.

Your response should aim for a minimum of 3-4 sentences and cover the following aspects:

1. Expand on the context provided by the hook and buildup, adding more details or background information to set the scene.
2. Incorporate persuasive language, rhetorical questions, or calls to action to encourage viewer engagement.
3. Consider including personal anecdotes, relatable experiences, or emotional appeals to connect with the audience.
4. Conclude with a strong closing statement that reinforces the main point or desired action.

### Hook:
"You shouldn't have 12 employees for a 50,000-month business. You should have one, you."

### Buildup:
Build Up:
"I'll give you examples. My agency model, absolute mess, so complicated. We had 12 employ

In [ ]:
pip install bert-score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manyli

In [ ]:
from bert_score import score
actual_bodies= train_dataset['Body']
# Calculate BERTScores
P, R, F1 = score(cands=generated_bodies, refs=actual_bodies, lang="en", verbose=True)

# Calculate and print the average scores
mean_precision = P.mean().item()
mean_recall = R.mean().item()
mean_f1 = F1.mean().item()

print("Average Precision:", mean_precision)
print("Average Recall:", mean_recall)
print("Average F1 Score:", mean_f1)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 3.97 seconds, 8.56 sentences/sec
Average Precision: 0.8469383120536804
Average Recall: 0.8437687158584595
Average F1 Score: 0.845227062702179


## Validation on test_dataset

In [ ]:
generated_bodies1 = []
# Assuming test_dataset = {'hook': [...], 'buildup': [...], 'body': [...]}
for hook, buildup, actual_body in zip(test_dataset['Hook'], test_dataset['Build Up'], test_dataset['Body']):
    # Prepare the prompt
    prompt = alpaca_prompt.format(
        hook,  # Hook part
        buildup,  # Buildup part
        ""  # Leave the body part empty for generation
    )

    # Tokenize the inputs
    inputs = tokenizer(
        [prompt],
        return_tensors="pt"
    ).to("cuda")

    # Generate the output using the model
    generated_outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        no_repeat_ngram_size=5,
        use_cache=True,
        early_stopping=True
    )

    # Decode the generated text
    generated_body = tokenizer.decode(generated_outputs[0], skip_special_tokens=True)
    start = generated_body.find("### Body:") + len("### Body:")
    body_text = generated_body[start:].strip()  # Extract the body text
    generated_bodies1.append(body_text)

    # Print both generated and actual body for comparison
    print("Generated Body:", generated_body)
    print("Actual Body:", actual_body)
    print("\n")  # Adds a newline for better separation of entries


Generated Body: Below is a hook and a buildup that describes the context for a social media post. Based on this context, generate an engaging and elaborate body that captures the viewer's attention, provides relevant details, and encourages interaction.

Your response should aim for a minimum of 3-4 sentences and cover the following aspects:

1. Expand on the context provided by the hook and buildup, adding more details or background information to set the scene.
2. Incorporate persuasive language, rhetorical questions, or calls to action to encourage viewer engagement.
3. Consider including personal anecdotes, relatable experiences, or emotional appeals to connect with the audience.
4. Conclude with a strong closing statement that reinforces the main point or desired action.

### Hook:
"You need to be using this free editing pack for all your videos."

### Buildup:
"Motion backgrounds, sound effects, transitions, luts, font, Instagram border,"

### Body:
"I literally just made this vi

In [ ]:
from bert_score import score
actual_bodies1= test_dataset['Body']
# Calculate BERTScores
P, R, F1 = score(cands=generated_bodies1, refs=actual_bodies1, lang="en", verbose=True)

# Calculate and print the average scores
mean_precision = P.mean().item()
mean_recall = R.mean().item()
mean_f1 = F1.mean().item()

print("Average Precision:", mean_precision)
print("Average Recall:", mean_recall)
print("Average F1 Score:", mean_f1)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


calculating scores...
computing bert embedding.


  0%|          | 0/1 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 1.19 seconds, 7.55 sentences/sec
Average Precision: 0.8273319602012634
Average Recall: 0.8377459645271301
Average F1 Score: 0.8322226405143738


## Saving, loading finetuned models

In [ ]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.model',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

In [ ]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "You've been told by all the YouTube gurus that scaling your agency is the fastest path to 20, 30, or even a hundred thousand a month in profit.", # hook
        "Well, the truth is that all the YouTube gurus are actually using a completely different model. And if you aren't implementing this in 2024 you're going to get left behind.", # buildup
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 512, no_repeat_ngram_size=5, use_cache = True, early_stopping=True)
print(tokenizer.batch_decode(outputs)[0])


==((====))==  Unsloth: Fast Mistral patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


<s> Below is a hook and a buildup that describes the context for a social media post. Based on this context, generate an engaging and elaborate body that captures the viewer's attention, provides relevant details, and encourages interaction.

Your response should aim for a minimum of 3-4 sentences and cover the following aspects:

1. Expand on the context provided by the hook and buildup, adding more details or background information to set the scene.
2. Incorporate persuasive language, rhetorical questions, or calls to action to encourage viewer engagement.
3. Consider including personal anecdotes, relatable experiences, or emotional appeals to connect with the audience.
4. Conclude with a strong closing statement that reinforces the main point or desired action.

### Hook:
You've been told by all the YouTube gurus that scaling your agency is the fastest path to 20, 30, or even a hundred thousand a month in profit.

### Buildup:
Well, the truth is that all the YouTube gurus are actual

## Deployment on Web Portal

In [ ]:
!pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.9/315.9 kB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 11.8 MB/

In [ ]:
import gradio as gr

def generate_text(hook, buildup):
    # Constructing the prompt correctly
    prompt = f"### Hook:\n{hook}\n### Buildup:\n{buildup}\n### Body:"
    # Encode the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    # Generate text from the model
    outputs = model.generate(**inputs, max_length=512, no_repeat_ngram_size=5,use_cache = True, early_stopping=True)
    # Decode the output
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract the body text
    start = generated_text.find("### Body:") + len("### Body:")
    if start > len("### Body:"):  # Ensure "### Body:" was found
        body_text = generated_text[start:].strip()  # Extract text following "### Body:"
    else:
        body_text = "No body generated. Check input or model."  # Fallback message

    return body_text

# Setup the Gradio interface
iface = gr.Interface(
    fn=generate_text,
    inputs=[
        gr.Textbox(lines=2, placeholder="Enter Hook Here...", label="Hook"),
        gr.Textbox(lines=5, placeholder="Enter Buildup Here...", label="Buildup")
    ],
    outputs=gr.Textbox(label="Generated Script"),
    title="Social Media Post Creator",
    description="Enter a compelling Hook and detailed Buildup to generate a social media post that captivates and engages your audience. This tool helps you develop intriguing posts that resonate with viewers, encouraging interaction and wider reach.",

)

iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0fb6d08d594f457122.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
